In [1]:
from bonsai.data_loaders import load_data
from bonsai.net_vert import Net
from bonsai.trainers import *

%load_ext autoreload
%autoreload 2

In [2]:
dataset = 'CIFAR10'
classes = 10
gpu_space = 8.2
hypers = {
    'scale':7,
    'batch_size':16,
    'reductions':{'init':1,'final':5},
    'spacing':3,
    'nodes':4,
    'lr_schedule':
        {'lr_min': 0,
         'lr_max': 1e-3,
         't_0': 600,
         't_mult': 1},
    'epochs':600,
    'drop_prob':.25,
}
schedule = {'cycle_len':4,
            'transition_after':2,
            'n_cycles':4}
start_size=1

In [3]:
data, data_shape = load_data(hypers['batch_size'], dataset)
print("Edges per cell:",cell_cnx(hypers['nodes']))

# init model
model = Net(dim=data_shape, 
            classes=classes, 
            scale=hypers['scale'],
            reductions=start_size,
            spacing=hypers['spacing'],
            nodes=hypers['nodes'],
            prune=True,
            auxiliary=True)

size, overflow = size_test(model, data)
print(model)
init_or_finish=True

Edges per cell: 24.0
===================== NETWORK =====================
              Saul McFadden Carpathia              
Initializer         :                    640 params
Cell 0  (Normal)    : 128 x 32 ,     508,729 params
Cell 1  (Normal)    : 128 x 32 ,     508,730 params
Cell 2  (Reduction) : 256 x 16 ,   2,000,955 params
 ↳ Classifier       :                655,370 params
Total               :              3,674,424 params



In [ ]:
tries=1
transition = schedule['cycle_len']*schedule['transition_after']
comp_lambdas = TransitionDict(
    {0: None, transition: {'edge': .33*tries, 'input': .1*tries}}
)
lr_schedule = {'lr_min': hypers['lr_schedule']['lr_min'],
               'lr_max': hypers['lr_schedule']['lr_max']/10,
               't_0': schedule['cycle_len']*schedule['n_cycles'],
               't_mult': 1}   
epochs=1
comp_ratio = .8
init_or_finish = full_train(
                model, data,
                lr_schedule=lr_schedule,
                epochs=epochs,
                drop_prob=hypers['drop_prob'],
                half=False,

            );

=== Training Saul McFadden Carpathia ===
Starting at 2019-09-10 19:30:50.951115
0: 34.00MiB
1: 2.04GiB
2: 3.90GiB
